### Financial Datasets

- Yahoo Finance - http://finance.yahoo.com
- Google Finance - https://www.google.com/finance
- QuantQuote - https://www.quantquote.com (S&P500 EOD data only)
- EODData - http://eoddata.com (requires registration)

### Historical Data Structure

- Exchanges - What is the ultimate original source of the data?
- Vendor - Where is a particular data point obtained from?
- Instrument/Ticker - The ticker/symbol for the equity or index, along with corporate information of the underlying firm or fund.
- Price - The actual price for a particular security on a particular day.
- Corporate Actions - The list of all stock splits or dividend adjustments (this may lead to one or more tables), necessary for adjusting the pricing data.
- National Holidays - To avoid mis-classifying trading holidays as missing data errors, it can be useful to store national holidays and cross-reference.

### MySQL for Securities Masters

#### Installing MySQL

sudo apt-get install mysql-server

#### Configuring MySQL

mysql -u root -p
mysql> CREATE DATABASE securities_master;
mysql> USE securities_master;

mysql> CREATE USER ’sec_user’@’localhost’ IDENTIFIED BY ’password’;
mysql> GRANT ALL PRIVILEGES ON securities_master.* TO ’sec_user’@’localhost’;
mysql> FLUSH PRIVILEGES;

#### Schema Design for EOD Equities

In [6]:
# Manipulação de banco de dados
import sqlite3                                 

# conexão
cnn = sqlite3.connect('database/dbproject.db') 

# Carregamos a extensão SQL para uso de comandos SQL direto no notebook
%load_ext sql

# Definimos o banco de dados caminho
%sql sqlite:///database/dbproject.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [17]:
%%sql
CREATE TABLE  exchange (id  int NOT NULL AUTO_INCREMENT,
                        abbrev varchar(32) NOT NULL,
                        name varchar(255) NOT NULL,
                        city varchar(255) NULL,
                        country varchar(255) NULL,
                        currency varchar(64) NULL,
                        timezone_offset time NULL,
                        created_date datetime NOT NULL,
                        last_updated_date datetime NOT NULL,
  PRIMARY KEY ( id )
)
ENGINE=InnoDB AUTO_INCREMENT=1 DEFAULT CHARSET=utf8;

 * sqlite:///database/dbproject.db
(sqlite3.OperationalError) near "AUTO_INCREMENT": syntax error
[SQL: CREATE TABLE  exchange ( idd  int NOT NULL AUTO_INCREMENT,
                        abbrev varchar(32) NOT NULL,
                        name varchar(255) NOT NULL,
                        city varchar(255) NULL,
                        country varchar(255) NULL,
                        currency varchar(64) NULL,
                        timezone_offset time NULL,
                        created_date datetime NOT NULL,
                        last_updated_date datetime NOT NULL,
  PRIMARY KEY ( idd )
)]
(Background on this error at: https://sqlalche.me/e/14/e3q8)


In [ ]:
%%sql
CREATE TABLE data_vendor (id int NOT NULL AUTO_INCREMENT,
                          name varchar(64) NOT NULL,
                          website_url varchar(255) NULL,
                          support_email varchar(255) NULL,
                          created_date datetime NOT NULL,
                          last_updated_date datetime NOT NULL,
    PRIMARY KEY ( id )
)
ENGINE=InnoDB AUTO_INCREMENT=1 DEFAULT CHARSET=utf8;

In [ ]:
%%sql
CREATE TABLE symbol (id int NOT NULL AUTO_INCREMENT,
                     exchange_id int NULL,
                     ticker varchar(32) NOT NULL,
                     instrument varchar(64) NOT NULL,
                     name varchar(255) NULL,
                     sector varchar(255) NULL,
                     currency varchar(32) NULL,
                     created_date datetime NOT NULL,
                     last_updated_date‘ datetime NOT NULL,
    PRIMARY KEY ( id ),
    KEY index_exchange_id ( exchange_id )
) 
ENGINE=InnoDB AUTO_INCREMENT=1 DEFAULT CHARSET=utf8;


In [ ]:
%%sql
CREATE TABLE daily_price (id int NOT NULL AUTO_INCREMENT,
                          data_vendor_id int NOT NULL,
                          symbol_id int NOT NULL,
                          price_date datetime NOT NULL,
                          created_date datetime NOT NULL,
                          last_updated_date datetime NOT NULL,
                          open_price decimal(19,4) NULL,
                          high_price decimal(19,4) NULL,
                          low_price decimal(19,4) NULL,
                          close_price decimal(19,4) NULL,
                          adj_close_price decimal(19,4) NULL,
                          volume bigint NULL,
    PRIMARY KEY ( id ),
    KEY index_data_vendor_id ( data_vendor_id ),
    KEY index_symbol_id ( symbol_id )
) 
ENGINE=InnoDB AUTO_INCREMENT=1 DEFAULT CHARSET=utf8;

#### Connecting to the Database

! sudo apt-get install libmysqlclient-dev
! pip install mysqlclient

#### Using an Object-Relational Mapper

pip install requests
pip install beautifulsoup4

In [ ]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
# insert_symbols.py


from __future__ import print_function
import datetime
from math import ceil

import bs4
import MySQLdb as mdb
import requests

def obtain_parse_wiki_snp500():
    """
    Download and parse the Wikipedia list of S&P500
    constituents using requests and BeautifulSoup.
    Returns a list of tuples for to add to MySQL.
    """

    # Stores the current time, for the created_at record
    now = datetime.datetime.utcnow()

    # Use requests and BeautifulSoup to download the
    # list of S&P500 companies and obtain the symbol table
    response = requests.get("http://en.wikipedia.org/wiki/List_of_S%26P_500_companies")
    soup = bs4.BeautifulSoup(response.text)
    
    # This selects the first table, using CSS Selector syntax
    # and then ignores the header row ([1:])
    symbolslist = soup.select('table')[0].select('tr')[1:]

    # Obtain the symbol information for each
    # row in the S&P500 constituent table
    symbols = []
    for i, symbol in enumerate(symbolslist):
        tds = symbol.select('td')
        symbols.append((tds[0].select('a')[0].text, # Ticker
                        'stocks',
                        tds[1].select('a')[0].text, # Name
                        tds[3].text, # Sector
                        'USD', now, now))
    
    return symbols

def insert_snp500_symbols(symbols):
    """
    Insert the S&P500 symbols into the MySQL database.
    """
    # Connect to the MySQL instance
    db_host = 'localhost'
    db_user = 'sec_user'
    db_pass = 'password'
    db_name = 'securities_master'
    
    con = mdb.connect(host=db_host, 
                      user=db_user, 
                      passwd=db_pass, 
                      db=db_name)
    
    # Create the insert strings
    column_str = """ticker, instrument, name, sector,
                    currency, created_date, last_updated_date
                    """
    
    insert_str = ("%s, " * 7)[:-2]
    final_str = "INSERT INTO symbol (%s) VALUES (%s)" % \
    (column_str, insert_str)
    # Using the MySQL connection, carry out
    # an INSERT INTO for every symbol
    with con:
        cur = con.cursor()
        cur.executemany(final_str, symbols)

if __name__ == "__main__":
    symbols = obtain_parse_wiki_snp500()
    insert_snp500_symbols(symbols)
    print("%s symbols were successfully added." % len(symbols))
    
